In [1]:
from modeling.learner import get_preds
from modeling import ASTPretrained
from modeling.dataset import get_loader
from modeling.utils import parse_config
from modeling.metrics import *

import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
%matplotlib inline

NameError: name 'Dataloader' is not defined

In [ ]:
CONFIG_PATH = "../configs/config.yaml"
config = parse_config(CONFIG_PATH)

In [ ]:
valid_dl = get_loader(config, subset="valid")
test_dl = get_loader(config, subset="test")

In [1]:
weight_list = ["bpmsync_2", "no_sync1"]

base_model = ASTPretrained(n_classes=11, download_weights=False)

NameError: name 'ASTPretrained' is not defined

In [ ]:
def evaluate(weight_list: list, base_model: nn.Module):
    # Create empty lists to store the accuracy and F1 score for each weight on the validation and test sets
    val_accs, val_f1s, test_accs, test_f1s = [], [], [], []

    # Loop through each weight and calculate accuracy and F1 score on the validation and test sets
    weights = [f"../weights/{weight}.pth" for weight in weight_list]
    
    for w in weights:
        # Make predictions on the validation set using the current weight
        base_model.load_state_dict(torch.load(w))
        preds, targets = get_preds(valid_dl, base_model, device="cuda")

        # Calculate accuracy and F1 score on the validation set
        thresholds = optimize_accuracy(preds, targets)
        
        val_acc = hamming_score(preds, targets, thresholds)
        val_f1 = mean_f1_score(preds, targets, thresholds)

        # Make predictions on the test set using the current weight
        preds, targets = get_preds(test_dl, base_model, device="cuda")

        # Calculate accuracy and F1 score on the test set
        test_acc = hamming_score(preds, targets, thresholds)
        test_f1 = mean_f1_score(preds, targets, thresholds)

        # Append the results to the corresponding lists
        val_accs.append(val_acc)
        val_f1s.append(val_f1)
        test_accs.append(test_acc)
        test_f1s.append(test_f1)

    # Create a pandas dataframe to display the results
    df = pd.DataFrame({
        'Weight': weight_list,
        'Val Acc': val_accs,
        'Val F1': val_f1s,
        'Test Acc': test_accs,
        'Test F1': test_f1s
    })

    # Display the dataframe
    return df